# Beach Resources OW VisTool/STHLM office

In [62]:
## set up Dash with classes for all of the visual components of the application
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from geopy.geocoders import Nominatim 
import folium
from itertools import count
from datetime import datetime
import itertools
import plotly 
import plotly.plotly as py
from plotly.graph_objs import *

#output_notebook() ## run all polts within jopyter notebook 

In [63]:
# Reading '.xlsx' file 
def import_excel(xlxs_file, sheet_num,sheet_idx=False):
    xlsx = pd.ExcelFile(xlxs_file)
    sheet1 = xlsx.parse(sheet_num)
    if sheet_idx:
        sheet1.index = sheet1['Name']
        new_df_beach_0 = sheet1.iloc[:]
        return new_df_beach_0
    else:
        new_df_beach_0 = sheet1.iloc[:]
        return new_df_beach_0

new_df_beach = import_excel(
    r'C:\Users\omar.hazim\Desktop\Copy of Beach Work HTMLversion.xlsx', sheet_num=0, sheet_idx=True
) #, sheet_idx='Name'
new_df_beach.head()

,Name,Beach,Location,Responsible,Case code,Priority,Client,Project,Deadline/End date,Days until DL,Status,Comment
Name,,,,,,,,,,,,
Alexander,Alexander,yes,Stockholm,NaN,Carl - FSP31001,Internal - Firm building,Bank A,NaN,2018-04-14,88,Ongoing,NaN
Anders,Anders,yes,London,NaN,Sean - FSP03201,Internal - Business development,Bank B,NaN,2018-04-14,88,Ongoing,NaN
Andreas,Andreas,yes,Germany,NaN,Bo - N/A,Project - Soft start,Bank A,NaN,2018-04-14,88,Ongoing,NaN
Artin,Artin,no,France,NaN,Nader - FSP11101,Project - Delivery risk / WLB,Bank B,NaN,2018-04-14,88,NaN,NaN
Axel,Axel,yes,Italy,NaN,Martin - FSP32601,Project - General help,Bank A,NaN,2018-04-14,88,Ongoing,NaN


In [64]:
# Building a pivot table to plot the beach resources
#new_df_beach['Deadline/End date'].value_counts()
#
## 1D counting given one feature to DF
uniq, count = np.unique(new_df_beach['Priority'], return_counts=True)
_1D_count = dict(zip(uniq, count))

## 2D counting given 2 features to DF
df_grop = new_df_beach.groupby(['Beach','Deadline/End date', 'Name'])
#list(zip(*df_grop)).sum()
plt_df = df_grop.count()
new_plt_df = pd.DataFrame(np.array(list(zip(*df_grop.Beach))[0]), columns=['Beach','Deadline/End date', 'Name'])
#pd.to_datetime(new_plt_df[1])
new_plt_df.replace(pd.to_datetime(new_plt_df['Deadline/End date'], format='%Y%m%d'), new_df_beach['Deadline/End date'])
#plt.bar(x=plt_df['Days until DL'], y=plt_df['Deadline/End date'])
counts = new_plt_df['Deadline/End date'].value_counts()
beach = new_plt_df['Beach'].value_counts()

c:\users\omar.hazim\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:9: FutureWarning:

'Name' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version



In [65]:
################ plotting using bokeh##############
#source = ColumnDataSource(data = dict(x=counts.index,count=counts.values))
#data = pd.DataFrame(counts)
#output_file("test.html")
##Creating plots using bokeh
date = [datetime.strftime(x, format='%Y-%m-%d') for x in counts.index]
source = ColumnDataSource(data=dict(x_1=date, counts=list(counts.values)))
#p = figure()
p = figure(x_range=date,plot_height=250,toolbar_location=None, title="Beach resources Deadline")
p.background_fill_color = "whitesmoke"
p.background_fill_alpha = .9
p.vbar(x='x_1',width=.4, top='counts', source=source, legend="Beach Res. count")
p.border_fill_color = "whitesmoke"
p.min_border_left = 10
p.xgrid.grid_line_color = None
p.ygrid.grid_line_alpha = 0.9
p.ygrid.grid_line_dash = [6, 4]
show(p)
################ plotting using matplotlib##############
## set up a plot frame 
#fig, axs = plt.subplots(1,2, figsize=(13,5))
#for ax in axs:
   # ax.tick_params(color='white', labelcolor='white')
    #for spine in ax.spines.values():
        #spine.set_edgecolor('white')
#axs[0].bar(counts.index, counts.values)

#axs[1].pie(new_df_beach['Priority'].value_counts().values, 
        #labels=new_df_beach['Priority'].value_counts().index)




c:\users\omar.hazim\appdata\local\programs\python\python35\lib\site-packages\bokeh\core\json_encoder.py:80: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.



In [100]:
# GeoSpotting for all resource 
def poptext_2(df,location_col,name_col, x):
    """
    Supporting func to show info for each marker on the map
        Inputs:
        _______________
        - df: DataFrame
            dataset as a dataframe 
        - location_col: str
            location column within dataset df (e.g. city)
        - name_col: str
            names column within dataset df 
        - x: str
           location iterator (in case of run this func with df.apply)
        Outputs:
        ______________
        set of int and str for aggregations from main dataset df
    """
    new_df = new_df_beach[[location_col, name_col]].groupby([location_col,name_col]).count()
    nme_lst = []
    for item in new_df.index:
        if item[0] == x:
            nme_lst.append(item[1])
    df = pd.DataFrame({'city': x, 'Names':nme_lst})
    
    #'Country : {0} \n Num of resources : {1} \n Names : {2}'.format([x],[df[location_col].value_counts()[x]],nme_lst)
    return df #pd.DataFrame('city':x,[df[location_col].value_counts()[x]],nme_lst)

## Creat a func to count number of resources per country
def poptext_1(df,location_col,name_col, x):
    """
    Supporting func to show info for each marker on the map
        Inputs:
        _______________
        - df: DataFrame
            dataset as a dataframe 
        - location_col: str
            location column within dataset df (e.g. city)
        - name_col: str
            names column within dataset df 
        - x: str
           location iterator (in case of run this func with df.apply)
        Outputs:
        ______________
        set of int and str for aggregations from main dataset df
    """
    #info_df = pd.DataFrame()
    new_df = new_df_beach[[location_col, name_col]].groupby([location_col,name_col]).count()
    nme_lst = []
    for item in new_df.index:
        if item[0] == x:
            nme_lst.append(item[1])
            
    return 'Country : {0} \n Num of resources : {1} \n Names : {2}'.format([x],[df[location_col].value_counts()[x]],nme_lst)

## Creat a map 
## creating latitude/longtitude func
def PlotPoint(df, location_col, name_col):
    """
    Plotting coordinations points an a map
    Inputs:
    _______________
      - df: dataframe 
          the inputs dataset
      - location_col: str
            location column within dataset df (e.g. city)
        - name_col: str
            names column within dataset  
    """
    # converting location to a coordinate equivalents
    ResMap = folium.Map(prefer_canvas=False)
    geolocator = Nominatim()
    LongLat = df[location_col].apply(
    lambda x: folium.Marker(
        location=[geolocator.geocode(x, timeout=15).latitude, geolocator.geocode(x, timeout=15).longitude],
        popup=poptext_1(df, location_col,name_col, x), icon=folium.Icon(color='red',icon='info-sign', angle=10)).add_to(ResMap)
    if x == 'Stockholm' else folium.Marker(
        location=[geolocator.geocode(x, timeout=15).latitude, geolocator.geocode(x, timeout=15).longitude],
        popup=poptext_1(df, location_col,name_col, x), icon=folium.Icon(icon='info-sign')).add_to(ResMap)
    )

    #Set the zoom to the maximum possible
    ResMap.fit_bounds(ResMap.get_bounds())
    return ResMap


    # Plotting coordinations on the map
    #folium.CircleMarker(location=[geolocator.geocode(x, timeout=15).latitude, geolocator.geocode(x, timeout=15).longitude]).add_to(ResMap)



In [67]:
PlotPoint(new_df_beach, 'Location', 'Name')


c:\users\omar.hazim\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:22: FutureWarning:

'Name' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version



## Appex

In [68]:
#df_location = pd.DataFrame(list(new_df_beach.groupby(['Location']).Location))
#df_location.set_index(df_location[0], inplace=True)
#np.array(list(zip(*new_df_beach.groupby(['Location', 'Name']).Location)))[0]
#new_df_beach['Location_eql_sthlm'] = np.where(new_df_beach['Location']=="Stockholm", True, False)

###  - Beach resources visualization: GeoMap

In [101]:
############################ Plotting using Plotly & Dash ##########################################
## Running credentials 
plotly.tools.set_credentials_file(username='omar.hazim', api_key='UnoUlwMh8vwec5ezDXzB')
mapbox_access_token = 'pk.eyJ1Ijoib21hcmhhemltIiwiYSI6ImNqY2s5cHk3MzNyZDEycm1tanV6c3pzdGUifQ.hLOK6z98WohsI19MmNBiHw'

## Converting location colunms to a cooredinations figures
geolocator = Nominatim()
Lat = list(itertools.chain(new_df_beach['Location'].apply(lambda x: geolocator.geocode(x, timeout=15).latitude)))
Long = list(itertools.chain(new_df_beach['Location'].apply(lambda x: [geolocator.geocode(x, timeout=15).longitude])))
cities = list(itertools.chain(new_df_beach['Location'].values))
#info_0 = list(itertools.chain(
    #new_df_beach['Location'] +' '+
    #new_df_beach['Name'] +' '+
    #new_df_beach['Beach'] +' '+  
    #new_df_beach['Deadline/End date'].astype(str))) 
info_1 = new_df_beach['Location'].apply(lambda i: poptext_2(new_df_beach, 'Location', 'Name', i))

## Map all coordinations
data_beach = Data([ 
    Scattermapbox(
        lat=Lat,
        lon=Long,
        mode='markers',
        marker=Marker(
            size=14
        ),
        text = info_1,
        hoverinfo = True
    )
])

layout_beach = Layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=Lat,
            lon=Long
        ),
        pitch=0,
        zoom=0
    ),
)

fig_beach = dict(data=data_beach, layout=layout_beach)
py.iplot(fig_beach, filename='Beach Resources Mapbox')


c:\users\omar.hazim\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:19: FutureWarning:

'Name' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version



TypeError:        Names       city
0  Alexander  Stockholm
1   Jennifer  Stockholm
2       Otto  Stockholm is not JSON serializable

c:\users\omar.hazim\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:22: FutureWarning:

'Name' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version



"Country : ['Stockholm'] \n Num of resources : [3] \n Names : ['Alexander', 'Jennifer', 'Otto']"

In [92]:
info_1 = new_df_beach['Location'].apply(lambda i: poptext_2(new_df_beach, 'Location', 'Name', i))


c:\users\omar.hazim\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:18: FutureWarning:

'Name' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version



In [99]:
info_1[1]

,Names,city
0,Anders,London
1,Johan,London
2,Paul,London
